# 第七章：打造聊天應用程式
## OpenAI API 快速入門

本筆記本改編自 [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)，其中包含可存取 [Azure OpenAI](notebook-azure-openai.ipynb) 服務的筆記本。

Python 的 OpenAI API 也能搭配 Azure OpenAI 模型使用，只需做些微調。想了解更多差異，請參考：[如何用 Python 在 OpenAI 與 Azure OpenAI 端點間切換](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# 概覽  
「大型語言模型是一種將文字映射到文字的函數。給定一段文字輸入，大型語言模型會嘗試預測接下來可能出現的文字」(1)。這份「快速入門」筆記本將帶領使用者認識高階的 LLM 概念、開始使用 AML 所需的核心套件、提示設計的簡介，以及幾個不同應用情境的簡短範例。


## 目錄

[總覽](../../../../07-building-chat-applications/python)  
[如何使用 OpenAI 服務](../../../../07-building-chat-applications/python)  
[1. 建立你的 OpenAI 服務](../../../../07-building-chat-applications/python)  
[2. 安裝](../../../../07-building-chat-applications/python)  
[3. 認證資訊](../../../../07-building-chat-applications/python)  

[使用情境](../../../../07-building-chat-applications/python)  
[1. 文字摘要](../../../../07-building-chat-applications/python)  
[2. 文字分類](../../../../07-building-chat-applications/python)  
[3. 產生新產品名稱](../../../../07-building-chat-applications/python)  
[4. 微調分類器](../../../../07-building-chat-applications/python)  

[參考資料](../../../../07-building-chat-applications/python)


### 建立你的第一個提示  
這個簡短的練習將帶你基本認識如何向 OpenAI 模型提交提示，來完成一個簡單的任務「摘要」。

**步驟**:  
1. 在你的 python 環境中安裝 OpenAI 函式庫  
2. 載入標準輔助函式庫，並設定你為 OpenAI 服務所建立的安全憑證  
3. 選擇一個適合你任務的模型  
4. 為模型建立一個簡單的提示  
5. 將你的請求提交給模型 API！


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. 尋找合適的模型  
GPT-3.5-turbo 或 GPT-4 模型能夠理解並產生自然語言。


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. 提示設計

「大型語言模型的神奇之處在於，透過在大量文本上訓練以最小化預測誤差，模型最終學會了對這些預測有用的概念。例如，它們學會了像是」(1)：

* 如何拼字
* 語法如何運作
* 如何改寫句子
* 如何回答問題
* 如何進行對話
* 如何用多種語言寫作
* 如何寫程式
* 等等

#### 如何控制大型語言模型  
「在所有輸入大型語言模型的方式中，最具影響力的就是文字提示」(1)。

大型語言模型可以透過幾種方式被提示產生輸出：

指令：告訴模型你想要什麼  
補全：引導模型補全你想要的開頭  
示範：用以下方式讓模型知道你要什麼：  
在提示中給幾個範例  
在微調訓練資料集中給數百或數千個範例」


#### 建立提示有三個基本原則：

**明確說明與示範**。無論是透過指令、範例，或兩者結合，都要讓模型清楚知道你要什麼。如果你想讓模型將一串項目按字母順序排列，或根據情感分類一段文字，就要明確示範你的需求。

**提供高品質資料**。如果你想建立分類器或讓模型遵循某種模式，請確保有足夠的範例。一定要仔細檢查你的範例——模型通常足夠聰明，可以看穿基本的拼字錯誤並給你回應，但它也可能認為這是有意為之，進而影響回應內容。

**檢查你的設定。** temperature 和 top_p 設定會影響模型產生回應時的確定性。如果你要的回應只有一個正確答案，這些值應該設低一點。如果你想要更多元的回應，可以設高一點。大家最常犯的錯誤，就是以為這些設定是在調整「聰明度」或「創意」。

來源：https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## 摘要文字  
#### 挑戰  
透過在一段文字後面加上「tl;dr:」來進行摘要。請注意，模型能夠在沒有額外指示的情況下執行多種任務。你可以嘗試使用比 tl;dr 更具描述性的提示詞，來調整模型的行為並自訂你想要的摘要方式(3)。

近期的研究顯示，先在大量文本語料上進行預訓練，再針對特定任務進行微調，可以大幅提升許多 NLP 任務和基準測試的表現。雖然這種方法在架構上通常與任務無關，但仍然需要數千甚至數萬筆的任務專屬微調資料集。相比之下，人類通常只需要幾個範例或簡單的指示，就能學會新的語言任務——這是目前 NLP 系統仍然難以做到的。在這裡，我們展示了擴大語言模型規模能大幅提升與任務無關的少量學習表現，有時甚至能與先前最先進的微調方法相媲美。

tl;dr


# 多種應用情境的練習題  
1. 摘要文本  
2. 分類文本  
3. 產生新產品名稱


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 分類文字  
#### 挑戰  
將項目分類到推論時提供的類別中。在以下範例中，我們在提示中同時提供了類別和要分類的文字（*playground_reference）。

客戶詢問：你好，我的筆記型電腦鍵盤有一個按鍵最近壞掉了，我需要更換：

分類後的類別：


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 產生新產品名稱
#### 挑戰
根據範例單字創造產品名稱。我們會在提示中加入關於要命名產品的資訊，同時提供類似的範例，讓你了解我們希望得到的命名模式。我們也將溫度值設得較高，以增加隨機性和創新性。

產品描述：家用奶昔機
種子詞：快速、健康、輕巧
產品名稱：HomeShaker、Fit Shaker、QuickShake、Shake Maker

產品描述：一雙能適合任何腳型的鞋子
種子詞：可調整、合腳、全適配


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# 參考資料  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio 範例](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [微調 GPT-3 進行文本分類的最佳實踐](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# 需要更多協助

[OpenAI 商業化團隊](AzureOpenAITeam@microsoft.com)


# 貢獻者
* Louis Li



---

**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。雖然我們力求準確，但請注意，自動翻譯可能包含錯誤或不精確之處。原始語言的文件應視為具權威性的來源。對於重要資訊，建議尋求專業人工翻譯。我們對因使用本翻譯而產生的任何誤解或誤釋不承擔任何責任。
